# Implement LSH from scratch

In this assignment, you will implement LSH from scratch and predict the labels of the test data. You will then verify the correctness of the your implementation using a "grader" function/cell (provided by us) which will match your implmentation.

The grader fucntion would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader function/cell.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you write and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% weightage in the final marks for this course.

## Reading the data from csv file

In [81]:
# Code to mount google drive in case you are loading the data from your google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [82]:
# Loading data from csv file
import pandas as pd
data_path = '/gdrive/MyDrive/PGD Assignment Notebooks/lsh_assignment_data.csv'

df = pd.read_csv(data_path)
df

FileNotFoundError: ignored

In [64]:
# Data Overiview
df['category'].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

### Creating Train and Test Datasets

Note that the labels for test data will not be present in the dataset and hence they are mentioned as NaN.

In [65]:
# The last 10 rows in the csv file are query points, so loading them into test data.
# And loading the reamining points to train_data for which labels are given.

train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [66]:
# For train_data here the labels are in the column named "category".
train_data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2210,politics,teens know little of politics teenagers ques...
2211,entertainment,lopez misses uk charity premiere jennifer lope...
2212,business,christmas shoppers flock to tills shops all ov...
2213,tech,progress on new internet domains by early 2005...


In [67]:
test_data

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


<h1>Points that the student has used:</h1>
<b>a. </b><l>The student has used what previously has been taught,i.e, to pre-process and clean the data, as has been taught earlier in Amazon Food Reviews' Data Cleaning Chapter. Due to cleaning data, the number of dimensions has gone down to 1783 , even after allowing max_feautures = 4000 in TfidfVectorizer(max_features=4000, min_df=10,ngram_range=(2,3)). </l><br>
<b>b. </b><l>The student at first implemented the LSH using 5 random planes(with dim = dim of words) and then according to the lectures, extended it to use 5 groups of planes , with each group consisting its own 5 random planes.</l><br>
<b>c. </b><l>The student has modified last step of getting 11 nearest neighbors,a little. In this step, when we take cosine similarity with the query point's neighbours, the one which has less value should be less preferred in voting and the one which has higher value of cosine similarity with the query point should be preferred. To achieve this, the student has modified the step of counting 11 nearest neighbors to counting the most similar neighbours with weights proportional to their similarity. </l><br>
<small> To facilitate using this weight, a constant is introduced while taking the cosine similarities of a query point to all the possible neighbors in a region <br> The student acknowledge that this introduction of constant is dependent on the decimal places of cosine values, which will depend on nature of data.This constant's function is to take a particular digit place to act as the weight.<br> In this example, this constant is equal to 10, which gives 90 percent acc and if the constant is changed to 100, the accuracy becomes 100. This is happening because for constant 10 the neighbors which had cosine values with the query point, similar as 0.09 and 0.02 are assigned weight 0 while the neighbor which has cosine 0.19 is assigned weight 1 . </small>





In [68]:
#@title for cleaning
from tqdm import tqdm
from collections import defaultdict
from bs4 import BeautifulSoup
import numpy as np

In [69]:
#@title cleaning
import re

def decontracted(phrase):
    
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    
    return phrase

In [70]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [71]:
#@title cleaning_training_data
from tqdm import tqdm
def text_cleaner(ser):
  cleaned_reviews = []
  for sentance in tqdm(ser.values):
      sentance = re.sub(r"http\S+", "", sentance)
      sentance = BeautifulSoup(sentance, 'lxml').get_text()
      sentance = decontracted(sentance)
      sentance = re.sub("\S*\d\S*", "", sentance).strip()
      sentance = re.sub('[^A-Za-z]+', ' ', sentance)
      sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
      cleaned_reviews.append(sentance.strip())
  return cleaned_reviews



In [72]:
clean_train_data=text_cleaner(train_data.text)

100%|██████████| 2215/2215 [00:01<00:00, 1134.50it/s]


<h3>Vector Representation of train_data :</h3>

In [73]:
#@title for vectorization of train_data
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import numpy as np
np.random.seed(0)

In [74]:
tf_idf_vect2 = TfidfVectorizer(max_features=4000, min_df=10,ngram_range=(2,3))
tf_idf_fit2 = tf_idf_vect2.fit(clean_train_data)
tf_idf_transform2 = tf_idf_fit2.transform(clean_train_data)
#tf_idf_transform2.shape







<h3>Creating Hash Table</h3>
<h4>Here, Hash table is a dictionary which contains 5 sub-dictionaries </h4>
<h5>What does a sub-dictionary hold? <br> Answer: keys representing 32 regions made by 5 random h-planes</h5>


In [75]:
#@title Five groups of Five Random hyper-planes each
#hash_tab1 is a group of 5 random hplanes and likewise hash_tab2 and so is hash_tab3 and others.
#each hash_tab consists 32 regions made by 5 random h-planes. 
range_all = list()

for i in range(32):       #a simple hack to generate 32 distinct keys for each hash_tab, which will represent 32 regions created by a group of 5 random h-planes
  n = bin(i)[2:].zfill(5)
  k = tuple(int(i) for i in n)
  range_all.append(k)
hash_tab1={k:[] for k in range_all}
hash_tab2 = {k:[] for k in range_all}
hash_tab3 = {k:[] for k in range_all}
hash_tab4 = {k:[] for k in range_all}
hash_tab5 = {k:[] for k in range_all}


<h2>Training the model on vectorised taining_data</h2>
<h4>Basic method of this training?</h4>
<h5>using dot pdt of train_data(2215,1783) and transpose of 5planes(1783,5)</h5>
<h6>What is the meaning of this dot product?</h6>
<h>Dot pdt of a given data point with a generated h-plane, signifies the signed distance of that point from that h-plane<br> In a nutshell, </h7>
<small><b>we are getting signed distance from each of the 5 planes for each point. </b></small>
<small><i>above process is repeated 5 times,as I am making 5 groups of 5 random h-planes</i><small>

In [76]:
pdt_group = []
group_of_planes = []


hash_group = {'hash_tab1':hash_tab1,'hash_tab2':hash_tab2,'hash_tab3':hash_tab3,'hash_tab4':hash_tab4,'hash_tab5':hash_tab5}
#
for i in range(5):
    five_planes = np.array([[i for i in np.random.normal(0,1,1783)] for j in range(5)])#1783 instead of 4000? Because of cleaning, train_data has reduced its feautures ans so to maintain consistency.
    group_of_planes.append(five_planes)
    pdt=tf_idf_transform2.dot(five_planes.T)
    pdt = np.where(pdt>=0,1,0)
    pdt_group.append(pdt)
    #print(pdt.shape)
#print(np.array(pdt_group).shape)
pdt_group = np.array(pdt_group)
group_of_planes = np.array(group_of_planes)
for i,k in zip(pdt_group,hash_group.keys()):
    #lst=[]
    for j,j_ele in enumerate(i):
        hash_group[k][tuple(j_ele)].append(j) 


<h3>The following two cells will be used for test_data,later:</h3>

In [77]:
#@title multiplying test_data with the group of planes
def hash_new(text):
    lst_arr=np.array([text.dot(i.T) for i in group_of_planes])
    return lst_arr

In [78]:
def nearest_nbrs(transform_sample,ala):
      nbr_arr=((transform_sample.dot(tf_idf_transform2[ala].T))*10) 
      return nbr_arr


<b>In above cell:multiplication of 10 is there to facilitate using weight of the neighbouring point in voting<br>weight has been tried to be kept proportional to the magnitude of cosine sim<br></b>
<small>multiplication with 10 to obtain the significant digit <br>
and this digit is treated as its weight.If this digit is 100, accuracy increases<br>
 to 100% as in case all the cosine multiplication of a text with a plane<br>
gives 0.00something.multiplication of 10 will not be a proper choice <br>
 this constant will depend on data, and might be a drawback[not often].</small>

## Custom Implementation

In [79]:
# Please implement this fucntion and write your code wherever asked. Do NOT change the code snippets provided by us.

import numpy as np

def predictLabels (test_data):
  """
  Given the test_data, return the labels for all the rows in the test data.
  Follow the step by step instructions mentioned above.
  """

  np.random.seed(0)

  ##############################################################
  ####  Write YOUR CODE BELOW  as per the above instructions ###
  ##############################################################
  
  
  
  result = []

  
  for i in range(test_data.shape[0]):  
      test_text = test_data['text'].iloc[[i]].tolist()
      transform_sample=tf_idf_fit2.transform(test_text)
      sample_tes=hash_new(transform_sample)   
      sample_tes = np.where(sample_tes>=0,1,0)
      new_lst1 = [np.array(hash_group['hash_tab1'][tuple(sample_tes[0][0])]),
                  np.array(hash_group['hash_tab2'][tuple(sample_tes[1][0])]),
                  np.array(hash_group['hash_tab3'][tuple(sample_tes[2][0])]),
                  np.array(hash_group['hash_tab4'][tuple(sample_tes[3][0])]),
                  np.array(hash_group['hash_tab5'][tuple(sample_tes[4][0])])
                ]
      each_res_avg = []                                    
      for i,it in enumerate(new_lst1):
          my_nbr = nearest_nbrs(transform_sample,it)
          index = my_nbr.nonzero()[1]
          weight = [math.floor(my_nbr[(0,i)]) for i in index]
          index = np.repeat(index,weight)                  #this is done so that the index which has higher cosine similar value are repeated accordingly,to ensure a fairer voting to an extent                                                 
          if index.size != 0:
              ser_nbrs=df.category[it[index]].value_counts()
              every_res=ser_nbrs[ser_nbrs==ser_nbrs.max()].sort_index().index[0]
              each_res_avg.append(every_res)
          else:
              pass
      d = defaultdict(int)
      for i in each_res_avg:
          d[i] += 1
      #print(d)
      ser= pd.Series(d)
      #print(ser)
      every=ser[ser==ser.max()].sort_index().index[0]
      
      result.append(every)
      
      


  return result
        



### Instructions:

  1. Read in the train_data.
  2. Vectorize train_data using sklearns built in tfidf vectorizer.
  3. Ignore unigrams and make use of both **bigrams & trigrams** and also limit the **max features** to **4000** and **minimum document frequency** to **10**.
  4. After the tfidf vectors are generated as mentioned above, next task is to generate random hyperplanes.
  5. Generate **5 random hyperplanes**. And generate the hyperplanes using a random normal distribution with **mean zero and variance 1**. 
  6. We have set the **numpy random seed to zero**, please do not change it. And then you can make use of **np.random.normal** to generate the vectors for hyperplanes.
  7. As mentioned in the course videos, compute the hash function and also the corresponding hash table for it.
  8. Once the hash table is generated now take in each of the query points from the test data.
  9. Vectorize those query points using the same tfidf vectorizer as mentioned above.
  10. Now use the hash function on this query point and fetch all the similar data points from the hashtable.
  11. Use cosine similarity to compute **11-Nearest Neighbours** from the list of data points obtained in the above step.
  12. Take a majority vote among the 11-Nearest Neighbours and predict the class label for the query point in the test data.
  13. **In case of a tie** in the obtained labels from nearest neighbours, you can pick a label after sorting all the labels **alphabetically**(A-Z), i.e. for example labels starting with A would get more preference than labels starting with Z.
  14. Repeat steps 9 to 13 for all the points in the test data and then finally return a list with all the predicted labels.
  15. Note that there are a total of 10 data points in the test data so the final list you return should be of length 10.
  16. Also note that the cosine similarity function should be written from scratch, you should not directly make use of existing libraries.
  17. Please use the formula of cosine similarity as explained in the course videos, you can make use of numpy or scipy to calculate dot or norm or transpose.

## Grader Cell

Please execute the following Grader cell to verify the correctness of your above implementation. This cell will print "Success" if your implmentation of the predictLabels() is correct, else, it will print "Failed". Make sure you get a "Success" before you submit the code in the classroom.

In [80]:
###########################################
## GRADER CELL: Do NOT Change this.
# This cell will print "Success" if your implmentation of the predictLabels() is correct and the accuracy obtained is above 80%.
# Else, it will print "Failed"
###########################################
import numpy as np

# Predict the labels using the predictLabels() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

# Calculating accuracy by comparing Y_grader and Y_custom
accuracy = np.sum(Y_grader==Y_custom) * 10

if accuracy >= 80:
  print("******** Success ********","Accuracy Achieved = ", accuracy,'%')
else:
  print("####### Failed #######","Accuracy Achieved = ", accuracy,'%')
  print("\nY_grader = \n\n", Y_grader)
  print("\n","*"*50)
  print("\nY_custom = \n\n", Y_custom)


******** Success ******** Accuracy Achieved =  90 %
